# 5章 顧客の退会を予測する１０本ノック

引き続き、スポーツジムの会員データを使って顧客の行動を分析していきます。  
３章では顧客の全体像を把握し、4章では数ヶ月利用している顧客の来月の利用回数の予測を行いました。   
ここでは、教師あり学習の分類を用いて、顧客の退会予測を取り扱います。

### ノック41：データを読み込んで利用データを整形しよう

In [1]:
!ls

'5章 顧客の退会を予測する１０本ノック_answer.ipynb'   customer_join.csv
'5章 顧客の退会を予測する１０本ノック.ipynb'	      use_log_months.csv


In [56]:
import pandas as pd
df_customer_join = pd.read_csv("./customer_join.csv")
df_use_log_months = pd.read_csv("./use_log_months.csv")


In [57]:
df_use_log_months.head()

,年月,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [58]:
# 当月と一ヶ月前の利用履歴のみのデータを作成
df_41_uselog = df_use_log_months.copy()

df_41_uselog["年月"] = pd.to_datetime(df_41_uselog["年月"], format="%Y%m")
df_41_uselog.set_index("年月", inplace=True)

g = df_41_uselog.groupby("customer_id")
tmp = list()
for customer, df in g:
    df["count_1"]= df["count"].shift()
    tmp.append(df.dropna())
uselog = pd.concat(tmp)


In [4]:
uselog.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 32650 entries, 2018-05-01 to 2019-03-01
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_id  32650 non-null  object 
 1   count        32650 non-null  int64  
 2   count_1      32650 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1020.3+ KB


In [60]:
uselog.head()

,customer_id,count,count_1
年月,,,
2018-05-01,AS002855,5,4.0
2018-06-01,AS002855,5,5.0
2018-07-01,AS002855,5,5.0
2018-08-01,AS002855,3,5.0
2018-09-01,AS002855,7,3.0


In [64]:
df_customer_join

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14,NaN,CA1,0,オールタイム,10500,通常,8.000000,8.0,8,8,0,2019-04-30,1
4188,HD246549,XXXXX,C01,F,2019-03-14,NaN,CA1,0,オールタイム,10500,通常,10.000000,10.0,10,10,0,2019-04-30,1
4189,GD037007,XXXXX,C03,M,2019-03-14,NaN,CA1,0,ナイト,6000,通常,8.000000,8.0,8,8,0,2019-04-30,1
4190,OA953150,XXXXX,C01,M,2019-03-14,NaN,CA1,0,オールタイム,10500,通常,11.000000,11.0,11,11,0,2019-04-30,1


### ノック42：退会前月の退会顧客データを作成しよう

In [61]:
# 退会した顧客データだけ取得
df_42_customer = df_customer_join[df_customer_join["is_deleted"]==1].copy()

# 退会前月
from dateutil.relativedelta import relativedelta
df_42_customer["end_date"] = pd.to_datetime(df_42_customer["end_date"])
df_42_customer["exit_date"] = df_42_customer["end_date"].apply(lambda x: x-relativedelta(months=1))

# uselog とマージ
df_42 = df_42_customer.merge(uselog.reset_index(), on="customer_id")
df_42 = df_42.sort_values(by=["customer_id", "年月"] )
df_42 = df_42.groupby("customer_id").last()
df_42.head()

,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,...,median,max,min,routine_flg,calc_date,membership_period,exit_date,年月,count,count_1
customer_id,,,,,,,,,,,,,,,,,,,,,
AS008805,XXXXX,C01,M,2018-06-07,2019-03-31,CA2,1,オールタイム,10500,入会費半額,...,4.0,8,1,1,2019-03-31,9,2019-02-28,2019-03-01,1,6.0
AS015746,XXXXX,C01,M,2017-12-01,2018-08-31,CA3,1,オールタイム,10500,入会費無料,...,3.0,4,1,0,2018-08-31,8,2018-07-31,2018-08-01,3,3.0
AS019120,XXXX,C01,M,2018-08-07,2018-10-31,CA1,1,オールタイム,10500,通常,...,5.0,8,3,0,2018-10-31,2,2018-09-30,2018-10-01,3,5.0
AS025956,XXXXXX,C01,M,2018-05-09,2019-02-28,CA2,1,オールタイム,10500,入会費半額,...,4.5,7,1,1,2019-02-28,9,2019-01-28,2019-02-01,3,1.0
AS029624,XXXXX,C01,M,2018-05-08,2019-01-31,CA2,1,オールタイム,10500,入会費半額,...,5.0,8,1,1,2019-01-31,8,2018-12-31,2019-01-01,2,5.0


In [13]:
#解答例ではなぜか"2018-5-31"に退会したひとが抜け落ちている。
#私が作ったDFから"2018-5-31"を除くと同じデータ（行数）になる
df_42[df_42["end_date"]!="2018-5-31"]


,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,...,median,max,min,routine_flg,calc_date,membership_period,prev_month_of_end_date,年月,count,count_1
customer_id,,,,,,,,,,,,,,,,,,,,,
AS008805,XXXXX,C01,M,2018-06-07,2019-03-31,CA2,1,オールタイム,10500,入会費半額,...,4.0,8,1,1,2019-03-31,9,2019-02-28,2019-03-01,1,6.0
AS015746,XXXXX,C01,M,2017-12-01,2018-08-31,CA3,1,オールタイム,10500,入会費無料,...,3.0,4,1,0,2018-08-31,8,2018-07-31,2018-08-01,3,3.0
AS019120,XXXX,C01,M,2018-08-07,2018-10-31,CA1,1,オールタイム,10500,通常,...,5.0,8,3,0,2018-10-31,2,2018-09-30,2018-10-01,3,5.0
AS025956,XXXXXX,C01,M,2018-05-09,2019-02-28,CA2,1,オールタイム,10500,入会費半額,...,4.5,7,1,1,2019-02-28,9,2019-01-28,2019-02-01,3,1.0
AS029624,XXXXX,C01,M,2018-05-08,2019-01-31,CA2,1,オールタイム,10500,入会費半額,...,5.0,8,1,1,2019-01-31,8,2018-12-31,2019-01-01,2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TS963433,XX,C02,M,2018-07-14,2018-12-31,CA1,1,デイタイム,7500,通常,...,4.5,6,1,0,2018-12-31,5,2018-11-30,2018-12-01,1,4.0
TS971314,XXXX,C03,M,2018-12-01,2019-01-31,CA3,1,ナイト,6000,入会費無料,...,5.5,7,4,1,2019-01-31,1,2018-12-31,2019-01-01,4,7.0
TS973306,XX,C01,M,2017-07-01,2018-07-31,CA2,1,オールタイム,10500,入会費半額,...,3.0,4,1,1,2018-07-31,12,2018-06-30,2018-07-01,1,2.0


### ノック43：継続顧客のデータを作成しよう

In [62]:
# 退会した顧客データだけ取得
#df_43_customer = df_customer_join[df_customer_join["is_deleted"]==0].copy()

df_43_customer = df_customer_join.copy()
df_43_customer = df_43_customer[df_43_customer["is_deleted"] == 0]

df_43 = pd.merge(uselog, df_43_customer, on = "customer_id", how="left")
df_43 = df_43.drop_duplicates(subset=["customer_id"])
# 
df_43 = df_43.dropna(subset=["name"])
df_43


,customer_id,count,count_1,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,AS002855,5,4.0,XXXX,C03,F,2016-11-01,NaN,CA1,0.0,ナイト,6000.0,通常,4.500000,5.0,7.0,2.0,1.0,2019-04-30,29.0
20,AS009373,4,3.0,XX,C01,F,2015-11-01,NaN,CA1,0.0,オールタイム,10500.0,通常,5.083333,5.0,7.0,3.0,1.0,2019-04-30,41.0
31,AS015233,7,7.0,XXXXX,C01,M,2018-05-13,NaN,CA2,0.0,オールタイム,10500.0,入会費半額,7.545455,7.0,11.0,4.0,1.0,2019-04-30,11.0
41,AS015315,3,6.0,XXXXX,C01,M,2015-07-01,NaN,CA1,0.0,オールタイム,10500.0,通常,4.833333,5.0,7.0,3.0,1.0,2019-04-30,45.0
52,AS015739,5,7.0,XXXXX,C03,M,2017-06-01,NaN,CA1,0.0,ナイト,6000.0,通常,5.583333,5.5,8.0,4.0,1.0,2019-04-30,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32610,TS995853,8,11.0,XXXX,C01,M,2019-02-08,NaN,CA1,0.0,オールタイム,10500.0,通常,9.500000,9.5,11.0,8.0,1.0,2019-04-30,2.0
32611,TS998593,9,9.0,XXXXX,C03,M,2018-09-01,NaN,CA1,0.0,ナイト,6000.0,通常,8.142857,8.0,9.0,7.0,1.0,2019-04-30,7.0
32617,TS999079,5,7.0,XXX,C03,M,2016-06-01,NaN,CA1,0.0,ナイト,6000.0,通常,4.916667,5.5,9.0,2.0,1.0,2019-04-30,34.0
32628,TS999231,4,6.0,XXXX,C01,M,2017-03-01,NaN,CA1,0.0,オールタイム,10500.0,通常,4.666667,5.0,8.0,1.0,1.0,2019-04-30,25.0


In [63]:
df_predict_data = pd.concat([df_42.reset_index(), df_43])
df_predict_data.head().T
    

,0,1,2,3,4
customer_id,AS008805,AS015746,AS019120,AS025956,AS029624
name,XXXXX,XXXXX,XXXX,XXXXXX,XXXXX
class,C01,C01,C01,C01,C01
gender,M,M,M,M,M
start_date,2018-06-07,2017-12-01,2018-08-07,2018-05-09,2018-05-08
end_date,2019-03-31 00:00:00,2018-08-31 00:00:00,2018-10-31 00:00:00,2019-02-28 00:00:00,2019-01-31 00:00:00
campaign_id,CA2,CA3,CA1,CA2,CA2
is_deleted,1.0,1.0,1.000000,1.0,1.000000
class_name,オールタイム,オールタイム,オールタイム,オールタイム,オールタイム
price,10500.0,10500.0,10500.000000,10500.0,10500.000000


### ノック44：予測する月の在籍期間を作成しよう

In [67]:
df_predict_data

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,...,median,max,min,routine_flg,calc_date,membership_period,exit_date,年月,count,count_1
0,AS008805,XXXXX,C01,M,2018-06-07,2019-03-31 00:00:00,CA2,1.0,オールタイム,10500.0,...,4.0,8.0,1.0,1.0,2019-03-31,9.0,2019-02-28,2019-03-01,1,6.0
1,AS015746,XXXXX,C01,M,2017-12-01,2018-08-31 00:00:00,CA3,1.0,オールタイム,10500.0,...,3.0,4.0,1.0,0.0,2018-08-31,8.0,2018-07-31,2018-08-01,3,3.0
2,AS019120,XXXX,C01,M,2018-08-07,2018-10-31 00:00:00,CA1,1.0,オールタイム,10500.0,...,5.0,8.0,3.0,0.0,2018-10-31,2.0,2018-09-30,2018-10-01,3,5.0
3,AS025956,XXXXXX,C01,M,2018-05-09,2019-02-28 00:00:00,CA2,1.0,オールタイム,10500.0,...,4.5,7.0,1.0,1.0,2019-02-28,9.0,2019-01-28,2019-02-01,3,1.0
4,AS029624,XXXXX,C01,M,2018-05-08,2019-01-31 00:00:00,CA2,1.0,オールタイム,10500.0,...,5.0,8.0,1.0,1.0,2019-01-31,8.0,2018-12-31,2019-01-01,2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32610,TS995853,XXXX,C01,M,2019-02-08,NaN,CA1,0.0,オールタイム,10500.0,...,9.5,11.0,8.0,1.0,2019-04-30,2.0,NaT,NaT,8,11.0
32611,TS998593,XXXXX,C03,M,2018-09-01,NaN,CA1,0.0,ナイト,6000.0,...,8.0,9.0,7.0,1.0,2019-04-30,7.0,NaT,NaT,9,9.0
32617,TS999079,XXX,C03,M,2016-06-01,NaN,CA1,0.0,ナイト,6000.0,...,5.5,9.0,2.0,1.0,2019-04-30,34.0,NaT,NaT,5,7.0
32628,TS999231,XXXX,C01,M,2017-03-01,NaN,CA1,0.0,オールタイム,10500.0,...,5.0,8.0,1.0,1.0,2019-04-30,25.0,NaT,NaT,4,6.0


In [54]:
df_customer_join

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14,NaN,CA1,0,オールタイム,10500,通常,8.000000,8.0,8,8,0,2019-04-30,1
4188,HD246549,XXXXX,C01,F,2019-03-14,NaN,CA1,0,オールタイム,10500,通常,10.000000,10.0,10,10,0,2019-04-30,1
4189,GD037007,XXXXX,C03,M,2019-03-14,NaN,CA1,0,ナイト,6000,通常,8.000000,8.0,8,8,0,2019-04-30,1
4190,OA953150,XXXXX,C01,M,2019-03-14,NaN,CA1,0,オールタイム,10500,通常,11.000000,11.0,11,11,0,2019-04-30,1


### ノック45：欠損値を除去しよう

### ノック46：文字列型の変数を処理できるように整形しよう

### ノック47：決定木を用いて退会予測モデルを作成してみよう

### ノック48：予測モデルの評価を行ない、モデルのチューニングをしてみよう

### ノック49：モデルに寄与している変数を確認しよう

### ノック50：顧客の退会を予測しよう